# 电商评分预测 - PyCaret回归实现

> 🎯 将评分视为连续值，使用回归方法预测

---

In [ ]:
# 环境准备
from pycaret.regression import *
import pandas as pd
import numpy as np

# 加载数据
data_path = "../../../supervised_learning/projects/04_ecommerce_rating_prediction/data/Womens_Clothing_E-Commerce_Reviews.csv"
data = pd.read_csv(data_path)

# 预处理
data = data.drop(['Unnamed: 0', 'Title', 'Review Text', 'Positive Feedback Count'], axis=1, errors='ignore')
data = data.dropna(subset=['Rating'])

# Rating保持为数值（回归任务）
data['Rating'] = data['Rating'].astype(float)

print(f"数据形状: {data.shape}")
print(f"\nRating统计:")
print(data['Rating'].describe())

In [ ]:
# 初始化回归环境
reg = setup(
    data=data,
    target='Rating',
    session_id=42,
    normalize=True,
    transformation=False,        # 不变换目标（评分1-5）
    verbose=True
)

In [ ]:
# 对比模型
best_model = compare_models(sort='MAE')  # 用MAE评估

In [ ]:
# 调优
tuned_model = tune_model(best_model, optimize='MAE')

In [ ]:
# 预测
predictions = predict_model(tuned_model)

# 后处理：四舍五入并限制在1-5
y_test = predictions['Rating']
y_pred_raw = predictions['prediction_label']
y_pred = np.clip(np.round(y_pred_raw), 1, 5).astype(int)

from sklearn.metrics import mean_absolute_error, mean_squared_error, accuracy_score

mae = mean_absolute_error(y_test, y_pred_raw)
rmse = np.sqrt(mean_squared_error(y_test, y_pred_raw))
accuracy = accuracy_score(y_test.astype(int), y_pred)

print(f"MAE: {mae:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"Accuracy (after rounding): {accuracy:.4f}")

In [ ]:
# 可视化
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 5))

# 预测vs实际
plt.subplot(1, 2, 1)
plt.scatter(y_test, y_pred_raw, alpha=0.3)
plt.plot([1, 5], [1, 5], 'r--')
plt.xlabel('Actual Rating')
plt.ylabel('Predicted Rating')
plt.title('Regression Predictions')

# 四舍五入后的混淆矩阵
plt.subplot(1, 2, 2)
from sklearn.metrics import confusion_matrix
import seaborn as sns
cm = confusion_matrix(y_test.astype(int), y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Rounded)')

plt.tight_layout()
plt.show()

## 总结

### 回归方法特点

**优势**:
- ✅ 考虑评分顺序
- ✅ MAE/RMSE更合理
- ✅ 可以预测中间值

**劣势**:
- ❌ 需要后处理（四舍五入）
- ❌ Accuracy可能略低于多分类

**完成！** ✅